In [1]:
import os
import sys
import re
import pandas as pd
import commands
import ast
import itertools
import pyspark.sql.functions
from pyspark.sql.functions import col
from datetime import datetime
from collections import Counter
import re
import numpy as np
#from pyspark.sql import SparkSession
spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
if not os.path.isdir(spark_home):
    raise ValueError('SPARK_HOME environment variable is not a directory')
if not os.path.isdir(os.path.join(spark_home, 'python')):
    raise ValueError('SPARK_HOME directory does not contain python')
sys.path.insert(0, os.path.join(spark_home, 'python'))
pylib_list = (item for item in os.listdir(os.path.join(spark_home, 'python/lib/'))
              if re.match(r'py4j-\d+(\.\d+)+-src\.zip\Z', item)
              )
try:
    py4j_file = max(pylib_list)
    py4j = os.path.join(spark_home, os.path.join('python/lib', py4j_file))
except ValueError:
    raise ValueError(
        'Could not find py4j'
    )
sys.path.insert(0, py4j)

spark_release_file = spark_home + "/RELEASE"
if os.path.exists(spark_release_file) and "Spark" in  open(spark_release_file).read():
    pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", " --master yarn-client \
                                         --executor-memory 4g --executor-cores 5 --driver-memory 16g"
                                        )
    if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
    os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

with open(os.path.join(spark_home, 'python/pyspark/shell.py')) as f:
    code = compile(f.read(), os.path.join(spark_home, 'python/pyspark/shell.py'), 'exec')
    exec(code)


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.2.6.5.0-292
      /_/

Using Python version 2.7.15 (default, May  1 2018 23:32:55)
SparkSession available as 'spark'.


In [14]:
import xlrd
new=pd.ExcelFile('/s/latluri1/Schema_changes/GCCT_EntitySchema_v2.4.1.xlsx')
old=pd.ExcelFile('/s/latluri1/Schema_changes/GCCT_EntitySchema_historic_v1.xlsx')
old_edited=pd.ExcelFile('/s/latluri1/Schema_changes/GCCT_EntitySchema_historic_v1_edited.xlsx')

In [4]:
a=new.parse("Account (Account)")
b=old.parse("Account (Account)")
x1= str(a[a["Logical Name"]=='accountid']["Type"].to_string(index=False)).lower()
x2= str(b[b["Logical Name"]=='accountid']["Type"].to_string(index=False)).lower()
if(x2!='series([], )' and x1!='series([], )'):
    if(x1!=x2):
            print "dtype changed"+"|"+dsc_req+"|"+str(sheet)+"|"+str(col)+"|"+x2+"|"+x1
elif(x2=='series([], )' and x1!='series([], )'):
    print "New column added"+"|"+dsc_req+"|"+str(sheet)+"|"+str(col)
#x3=str(set(list(old.parse(sheet)["Logical Name"])).difference(set(list(new.parse(sheet)["Logical Name"])))).lower()
elif(x2!='series([], )' and x1=='series([], )'):
    print "Column removed"+"|"+dsc_req+"|"+str(sheet)+"|"+str(col)
else:
    print "****CHECK THE CODE****"     
        

In [6]:
set(old_sheets).intersection(set(new.sheet_names))

{u'Account (Account)',
 u'Activity (ActivityPointer)',
 u'Address (CustomerAddress)',
 u'Appointment (Appointment)',
 u'Case (Incident)',
 u'Contact (Contact)',
 u'Engine (gcct_engine)',
 u'Entities list',
 u'Fax (Fax)',
 u'Goal (Goal)',
 u'Goal Metric (Metric)',
 u'Letter (Letter)',
 u'Material (gcct_material)',
 u'Note (Annotation)',
 u'Offer (gcct_offer)',
 u'Phone Call (PhoneCall)',
 u'Queue (Queue)',
 u'Queue Item (QueueItem)',
 u'Territory (Territory)',
 u'UII Action (UII_action)',
 u'UII Audit (UII_audit)',
 u'User (SystemUser)'}

In [8]:
common= set(old.sheet_names).intersection(set(new.sheet_names))
common.remove("Entities list")
columns_dtype_changed=[]
columns_added=[]
columns_removed=[]
tables_added=[]
tables_removed=[]

dsc_req="yes"
print "Action|requirement|table|column|old|new"
for sheet in common:
    a=new.parse(sheet)
    b=old.parse(sheet)
    gcct_table_name_1=re.findall('\((.*?)\)',sheet)
    gcct_table_name="".join(str(x) for x in gcct_table_name_1).lower()
    if gcct_table_name is '':
        gcct_table_name=str(sheet)
        #print sheet
    for col in set(list(a["Logical Name"])).union(set(list(b["Logical Name"]))):
        x1=str(a[a["Logical Name"]==col]["Type"].to_string(index=False)).lower()
        x2=str(b[b["Logical Name"]==col]["Type"].to_string(index=False)).lower()

        if(x2!='series([], )' and x1!='series([], )'):
            if(x1!=x2):
                    print "dtype changed"+"|"+dsc_req+"|"+str(sheet)+"|"+str(col)+"|"+x2+"|"+x1
                    columns_dtype_changed.append(col)
        elif(x2=='series([], )' and x1!='series([], )'):
            print "New column added"+"|"+dsc_req+"|"+str(sheet)+"|"+str(col)
            columns_added.append(col)
        #x3=str(set(list(old.parse(sheet)["Logical Name"])).difference(set(list(new.parse(sheet)["Logical Name"])))).lower()
        elif(x2!='series([], )' and x1=='series([], )'):
            print "Column removed"+"|"+dsc_req+"|"+str(sheet)+"|"+str(col)
            columns_removed.append(str(sheet+col))
        else:
            print "****Hypothetical scenario****"

x4=str(set(new.sheet_names).difference(set(old.sheet_names))).lower()
if(x4!='set([])'):
    print "tables added"+"|may be|"+ x4 
    tables_added.append(list(x4))


x5=str(set(old.sheet_names).difference(set(new.sheet_names))).lower()
if(x5!='set([])'):
    print "tables removed"+"|may be|"+ x5 
    tables_removed.append(list(x5))
                                                                                                        
                                                                          

Action|requirement|table|column|old|new
Column removed|yes|Address (CustomerAddress)|fmc_sourcesystem
Column removed|yes|Address (CustomerAddress)|fmc_lastupdated
New column added|yes|Address (CustomerAddress)|composite
Column removed|yes|Address (CustomerAddress)|fmc_emailaddress
Column removed|yes|Address (CustomerAddress)|fmc_lastupdatedby
Column removed|yes|Address (CustomerAddress)|fmc_sourceprimarykey
Column removed|yes|Address (CustomerAddress)|fmc_createdby
Column removed|yes|Address (CustomerAddress)|fmc_comments
Column removed|yes|Account (Account)|fmc_fcsdregioncode
New column added|yes|Account (Account)|gcct_onlineprofileusername
New column added|yes|Account (Account)|gcct_isinmarketname
New column added|yes|Account (Account)|gcct_usedassignedlincolndealerid
Column removed|yes|Account (Account)|fmc_bonus1
Column removed|yes|Account (Account)|fmc_servicealignmentlevel4
Column removed|yes|Account (Account)|fmc_73literenginecertificationindicator
Column removed|yes|Account (Ac

Column removed|yes|Account (Account)|fmc_vat_regn_num
Column removed|yes|Account (Account)|fmc_partssupplylocationid
Column removed|yes|Account (Account)|fmc_dealertype
New column added|yes|Account (Account)|gcct_loyaltyscore
Column removed|yes|Account (Account)|fmc_dealeractiveeffectivedate
Column removed|yes|Account (Account)|fmc_reactinvtransundercode
Column removed|yes|Account (Account)|fmc_dualdealershipdivisioncode
New column added|yes|Account (Account)|gcct_newassignedforddealerid
Column removed|yes|Account (Account)|fmc_dealerhours
New column added|yes|Account (Account)|gcct_dealercontact
New column added|yes|Account (Account)|onholdtime
New column added|yes|Account (Account)|gcct_fin
Column removed|yes|Account (Account)|fmc_otherdealercode
New column added|yes|Account (Account)|gcct_usedassignedlincolndealeridname
New column added|yes|Account (Account)|gcct_marketingemailpreference
Column removed|yes|Account (Account)|fmc_forddirectcomindicator
Column removed|yes|Account (Acco

New column added|yes|Account (Account)|msa_managingpartneridyominame
Column removed|yes|Account (Account)|fmc_dealerrole
Column removed|yes|Account (Account)|fmc_unformattedaddress2telephone3
New column added|yes|Account (Account)|gcct_mastermarketingemailpreference
Column removed|yes|Account (Account)|fmc_unformattedaddress2telephone1
New column added|yes|Account (Account)|gcct_marketingphonepreferenceupdatedon
Column removed|yes|Account (Account)|fmc_dealertechnicalzone
New column added|yes|Account (Account)|gcct_bestdaytimephonename
Column removed|yes|Account (Account)|fmc_transferallocation2
New column added|yes|Account (Account)|gcct_dealercontactname
New column added|yes|Account (Account)|gcct_ismasterpacode
Column removed|yes|Account (Account)|fmc_flmmultipoint
Column removed|yes|Account (Account)|fmc_invtransmitpa
Column removed|yes|Account (Account)|fmc_react_statusname
New column added|yes|Account (Account)|gcct_allmarketingscores
New column added|yes|Account (Account)|gcct_d

Column removed|yes|Phone Call (PhoneCall)|fmc_sourcemodifiedon
New column added|yes|Phone Call (PhoneCall)|stageid
Column removed|yes|Phone Call (PhoneCall)|fmc_excludeassignmentevaluation
Column removed|yes|Phone Call (PhoneCall)|fmc_sourceprimarykey
New column added|yes|Phone Call (PhoneCall)|gcct_parentphonecall
Column removed|yes|Phone Call (PhoneCall)|fmc_assessforvehiclename
Column removed|yes|Phone Call (PhoneCall)|fmc_callertype
New column added|yes|Phone Call (PhoneCall)|gcct_visibleondealerportal
New column added|yes|Phone Call (PhoneCall)|gcct_callfromcontactyominame
Column removed|yes|Phone Call (PhoneCall)|fmc_sourcemodifiedbyname
Column removed|yes|Phone Call (PhoneCall)|fmc_assessforvehicle
Column removed|yes|Phone Call (PhoneCall)|fmc_manualassignment
New column added|yes|Phone Call (PhoneCall)|gcct_agentextension
New column added|yes|Phone Call (PhoneCall)|gcct_recordingid
New column added|yes|Phone Call (PhoneCall)|sortdate
Column removed|yes|Phone Call (PhoneCall)|fm

New column added|yes|Case (Incident)|gcct_symptomclassificationname
New column added|yes|Case (Incident)|resolveby
New column added|yes|Case (Incident)|gcct_useridyominame
New column added|yes|Case (Incident)|gcct_datasourcesystemname
New column added|yes|Case (Incident)|modifiedbyexternalparty
Column removed|yes|Case (Incident)|fmc_scaddress
Column removed|yes|Case (Incident)|fmc_coursenumber
Column removed|yes|Case (Incident)|fmc_asgn_usr_excld_flgname
Column removed|yes|Case (Incident)|fmc_customerdelegateidname
Column removed|yes|Case (Incident)|fmc_issuetypeind
New column added|yes|Case (Incident)|gcct_bestcontactmethod
New column added|yes|Case (Incident)|gcct_symptomclassification
New column added|yes|Case (Incident)|gcct_contact
Column removed|yes|Case (Incident)|fmc_status
Column removed|yes|Case (Incident)|fmc_visitingowner
New column added|yes|Case (Incident)|gcct_customeraddressline2
New column added|yes|Case (Incident)|gcct_legacy_modifiedby
New column added|yes|Case (Inci

...

Column removed|yes|Case (Incident)|fmc_espcontractid
Column removed|yes|Case (Incident)|fmc_vehiclelineshortname
New column added|yes|Case (Incident)|gcct_followup
Column removed|yes|Case (Incident)|fmc_doccodeoritemnumber
New column added|yes|Case (Incident)|masteridname
New column added|yes|Case (Incident)|followuptaskcreated
New column added|yes|Case (Incident)|gcct_iswificapable
Column removed|yes|Case (Incident)|fmc_dealershipid
New column added|yes|Case (Incident)|gcct_caseclassificationname
Column removed|yes|Case (Incident)|fmc_reasoncategorycodename
Column removed|yes|Case (Incident)|fmc_campaignnumber
Column removed|yes|Case (Incident)|fmc_statusname
Column removed|yes|Case (Incident)|fmc_checknumber
Column removed|yes|Case (Incident)|fmc_sourcemodifiedby
Column removed|yes|Case (Incident)|fmc_curr_postn_name
New column added|yes|Case (Incident)|firstresponsesentname
Column removed|yes|Case (Incident)|fmc_actdatvalx
Column removed|yes|Case (Incident)|fmc_techhotlinename
New c

New column added|yes|Contact (Contact)|adx_identity_logonenabledname
New column added|yes|Contact (Contact)|gcct_newassigneddealeryominame
New column added|yes|Contact (Contact)|adx_publicprofilecopy
New column added|yes|Contact (Contact)|gcct_onlineprofileaccountregistrationdate
Column removed|yes|Contact (Contact)|fmc_optinoptouteffectivedate
New column added|yes|Contact (Contact)|gcct_newassigneddealername
Column removed|yes|Contact (Contact)|fmc_genericcontact
Column removed|yes|Contact (Contact)|fmc_customerstatusname
New column added|yes|Contact (Contact)|gcct_tag
New column added|yes|Contact (Contact)|adx_identity_emailaddress1confirmedname
Column removed|yes|Contact (Contact)|fmc_unformattedmobilephone
Column removed|yes|Contact (Contact)|fmc_custtitlecode
New column added|yes|Contact (Contact)|address3_name
New column added|yes|Contact (Contact)|gcct_smstextpreferenceupdatedbyidyominame
Column removed|yes|Contact (Contact)|fmc_newlincolnassigneddealername
New column added|yes|

New column added|yes|Contact (Contact)|gcct_newassignedforddealerid
New column added|yes|Contact (Contact)|address3_telephone1
New column added|yes|Contact (Contact)|address3_telephone3
New column added|yes|Contact (Contact)|address3_telephone2
New column added|yes|Contact (Contact)|adx_preferredlanguageid
Column removed|yes|Contact (Contact)|fmc_bestdaytimephone
New column added|yes|Contact (Contact)|gcct_likelihoodtoservice
Column removed|yes|Contact (Contact)|fmc_phoneoptouteffectivedate
New column added|yes|Contact (Contact)|gcct_usedassignedlincolndealeridname
New column added|yes|Contact (Contact)|gcct_marketingemailpreference
Column removed|yes|Contact (Contact)|fmc_clscoredefectorscr
Column removed|yes|Contact (Contact)|fmc_unformattedaddress2telephone2
New column added|yes|Contact (Contact)|address3_city
Column removed|yes|Contact (Contact)|fmc_unformattedaddress2telephone1
New column added|yes|Contact (Contact)|msdyn_disablewebtracking
New column added|yes|Contact (Contact)|a

Column removed|yes|Contact (Contact)|fmc_assigneddealermake
New column added|yes|Contact (Contact)|address3_postalcode
Column removed|yes|Contact (Contact)|fmc_dlrprs_seq_n
New column added|yes|Contact (Contact)|gcct_contactpreferenceupdatebyuseridyominame
New column added|yes|Contact (Contact)|gcct_usedassigneddelaeryominame
New column added|yes|Contact (Contact)|gcct_newassignedlincolndealerid
Column removed|yes|Contact (Contact)|fmc_clscorelikelihoodtosrvc
Column removed|yes|Contact (Contact)|fmc_newlincolnassigneddealeryominame
New column added|yes|Contact (Contact)|msa_managingpartnerid
Column removed|yes|Contact (Contact)|fmc_usedmercuryassigneddealereffectivedate
Column removed|yes|Contact (Contact)|fmc_usedmercuryassigneddealername
Column removed|yes|Contact (Contact)|fmc_employeeid
New column added|yes|Contact (Contact)|marketingonlyname
Column removed|yes|Contact (Contact)|fmc_stag1
New column added|yes|Contact (Contact)|adx_identity_twofactorenabledname
Column removed|yes|Co

Column removed|yes|Appointment (Appointment)|fmc_statusname
New column added|yes|Appointment (Appointment)|attachmenterrorsname
New column added|yes|Appointment (Appointment)|processid
Column removed|yes|Appointment (Appointment)|fmc_caseid
New column added|yes|Appointment (Appointment)|slainvokedid
New column added|yes|Appointment (Appointment)|gcct_reason
New column added|yes|Appointment (Appointment)|slainvokedidname
New column added|yes|Appointment (Appointment)|stageid
Column removed|yes|Appointment (Appointment)|fmc_sourceprimarykey
New column added|yes|Appointment (Appointment)|gcct_emailfollowup
New column added|yes|Appointment (Appointment)|gcct_providereason
New column added|yes|Appointment (Appointment)|gcct_dealerportal
Column removed|yes|Appointment (Appointment)|fmc_vinnumber
New column added|yes|Appointment (Appointment)|isunsafe
Column removed|yes|Appointment (Appointment)|fmc_sourcecreatedby
New column added|yes|Appointment (Appointment)|gcct_phonefollowup
New column a

In [19]:
common= set(old_edited.sheet_names).intersection(set(new.sheet_names))
common.remove("Entities list")
columns_dtype_changed=[]
columns_added=[]
columns_removed=[]
tables_added=[]
tables_removed=[]

dsc_req="yes"
print "Action|requirement|table|column|old|new"
for sheet in common:
    a=new.parse(sheet)
    b=old_edited.parse(sheet)
    gcct_table_name_1=re.findall('\((.*?)\)',sheet)
    gcct_table_name="".join(str(x) for x in gcct_table_name_1).lower()
    if gcct_table_name is '':
        gcct_table_name=str(sheet)
        #print sheet
    for col in set(list(a["Logical Name"])).union(set(list(b["Logical Name"]))):
        x1=str(a[a["Logical Name"]==col]["Type"].to_string(index=False)).lower()
        x2=str(b[b["Logical Name"]==col]["Type"].to_string(index=False)).lower()

        if(x2!='series([], )' and x1!='series([], )'):
            if(x1!=x2):
                    print "dtype changed"+"|"+dsc_req+"|"+str(sheet)+"|"+str(col)+"|"+x2+"|"+x1
                    columns_dtype_changed.append(col)
        elif(x2=='series([], )' and x1!='series([], )'):
            print "New column added"+"|"+dsc_req+"|"+str(sheet)+"|"+str(col)
            columns_added.append(col)
        #x3=str(set(list(old.parse(sheet)["Logical Name"])).difference(set(list(new.parse(sheet)["Logical Name"])))).lower()
        elif(x2!='series([], )' and x1=='series([], )'):
            print "Column removed"+"|"+dsc_req+"|"+str(sheet)+"|"+str(col)
            columns_removed.append(str(sheet+col))
        else:
            print "****Hypothetical scenario****"

x4=str(set(new.sheet_names).difference(set(old_edited.sheet_names))).lower()
if(x4!='set([])'):
    print "tables added"+"|may be|"+ x4 
    tables_added.append(list(x4))


x5=str(set(old_edited.sheet_names).difference(set(new.sheet_names))).lower()
if(x5!='set([])'):
    print "tables removed"+"|may be|"+ x5 
    tables_removed.append(list(x5))
                                                                                                        
                                                                          

Action|requirement|table|column|old|new
Column removed|yes|Address (CustomerAddress)|gcct_createdby
Column removed|yes|Address (CustomerAddress)|gcct_sourceprimarykey
Column removed|yes|Address (CustomerAddress)|gcct_sourcesystem
New column added|yes|Address (CustomerAddress)|composite
Column removed|yes|Address (CustomerAddress)|gcct_emailaddress
Column removed|yes|Address (CustomerAddress)|gcct_lastupdatedby
Column removed|yes|Address (CustomerAddress)|gcct_comments
Column removed|yes|Address (CustomerAddress)|gcct_lastupdated
New column added|yes|Account (Account)|gcct_onlineprofileusername
New column added|yes|Account (Account)|gcct_isinmarketname
Column removed|yes|Account (Account)|gcct_dealerhours
Column removed|yes|Account (Account)|gcct_sourcecreatedbyyominame
Column removed|yes|Account (Account)|gcct_totalutilizationpct2
Column removed|yes|Account (Account)|gcct_totalutilizationpct1
New column added|yes|Account (Account)|gcct_usedassignedlincolndealerid
Column removed|yes|Acc

Column removed|yes|Account (Account)|gcct_dualdealershipdivisioncode
Column removed|yes|Account (Account)|gcct_accountid
Column removed|yes|Account (Account)|gcct_firstservicepartsfeed
Column removed|yes|Account (Account)|gcct_fleetstatusname
Column removed|yes|Account (Account)|gcct_react_invupdatedate
New column added|yes|Account (Account)|gcct_newassignedforddealerid
New column added|yes|Account (Account)|gcct_dealercontact
New column added|yes|Account (Account)|onholdtime
New column added|yes|Account (Account)|gcct_fin
Column removed|yes|Account (Account)|gcct_reactstatusdescript
Column removed|yes|Account (Account)|gcct_cpaaccountname
Column removed|yes|Account (Account)|gcct_pacodetypename
New column added|yes|Account (Account)|gcct_usedassignedlincolndealeridname
New column added|yes|Account (Account)|gcct_marketingemailpreference
Column removed|yes|Account (Account)|gcct_sourcemodifiedbyyominame
Column removed|yes|Account (Account)|gcct_react_dspcustomerid
New column added|yes|

Column removed|yes|Account (Account)|gcct_golddchangedate
New column added|yes|Account (Account)|gcct_marketingmailpreference
New column added|yes|Account (Account)|createdbyexternalpartyname
Column removed|yes|Account (Account)|gcct_react_goldid
Column removed|yes|Account (Account)|gcct_lincolnnavigatorindicator
New column added|yes|Account (Account)|gcct_preferredlanguage
Column removed|yes|Account (Account)|gcct_react_rotempdisable
New column added|yes|Account (Account)|gcct_mastermarketingphonepreference
New column added|yes|Account (Account)|gcct_newassignedlincolndealerid
New column added|yes|Account (Account)|gcct_daylightsavingstimename
Column removed|yes|Account (Account)|gcct_react_usedvehpermdisable
Column removed|yes|Account (Account)|gcct_invtransmitpa
New column added|yes|Account (Account)|gcct_mastermarketingphonepreferencename
New column added|yes|Account (Account)|msa_managingpartnerid
Column removed|yes|Account (Account)|gcct_dpeslastupdatedate
New column added|yes|Ac

Column removed|yes|UII Audit (UII_audit)|gcct_interactionid
Column removed|yes|UII Audit (UII_audit)|gcct_interactionidname
Column removed|yes|UII Audit (UII_audit)|gcct_caseid
Column removed|yes|Case (Incident)|gcct_analystposition
New column added|yes|Case (Incident)|escalatedon
Column removed|yes|Case (Incident)|gcct_cust_control_treated_fname
New column added|yes|Case (Incident)|parentcaseidname
Column removed|yes|Case (Incident)|gcct_rolinenumber
New column added|yes|Case (Incident)|gcct_closurecomment
Column removed|yes|Case (Incident)|gcct_familyname
New column added|yes|Case (Incident)|gcct_symptomclassificationlevel1
Column removed|yes|Case (Incident)|gcct_fullpathidname
New column added|yes|Case (Incident)|gcct_symptomclassificationlevel3
New column added|yes|Case (Incident)|gcct_symptomclassificationlevel2
New column added|yes|Case (Incident)|gcct_symptomclassificationlevel4
New column added|yes|Case (Incident)|gcct_vehiclespecificationfullpathname
Column removed|yes|Case (I

New column added|yes|Case (Incident)|slainvokedid
Column removed|yes|Case (Incident)|gcct_level3id
Column removed|yes|Case (Incident)|gcct_checkissuedate
Column removed|yes|Case (Incident)|gcct_initialcontactstatusname
Column removed|yes|Case (Incident)|gcct_issue_open_s
Column removed|yes|Case (Incident)|gcct_sourcemodifiedbyname
Column removed|yes|Case (Incident)|gcct_reason
Column removed|yes|Case (Incident)|gcct_asgn_usr_excld_flg
New column added|yes|Case (Incident)|resolvebyslastatusname
New column added|yes|Case (Incident)|gcct_bodystylename
Column removed|yes|Case (Incident)|gcct_symptomidname
Column removed|yes|Case (Incident)|gcct_sourcemodifiedby
Column removed|yes|Case (Incident)|gcct_customerdelegateidyominame
Column removed|yes|Case (Incident)|gcct_itemdescription
New column added|yes|Case (Incident)|firstresponseslastatusname
New column added|yes|Case (Incident)|parentcaseid
Column removed|yes|Case (Incident)|gcct_level3idname
New column added|yes|Case (Incident)|gcct_ca

...

Column removed|yes|Letter (Letter)|gcct_manualassignment
Column removed|yes|Letter (Letter)|gcct_sourceprimarykey
New column added|yes|Letter (Letter)|processid
Column removed|yes|Letter (Letter)|gcct_pricelistname
New column added|yes|Letter (Letter)|slainvokedid
Column removed|yes|Letter (Letter)|gcct_privateflag
Column removed|yes|Letter (Letter)|gcct_morscaseid
Column removed|yes|Letter (Letter)|gcct_sourcemodifiedbyname
Column removed|yes|Letter (Letter)|gcct_last_upd_by
New column added|yes|Letter (Letter)|slainvokedidname
Column removed|yes|Letter (Letter)|gcct_customerlocaltime
Column removed|yes|Letter (Letter)|gcct_sourcemodifiedon
New column added|yes|Letter (Letter)|stageid
Column removed|yes|Letter (Letter)|gcct_sourcemodifiedby
Column removed|yes|Letter (Letter)|gcct_documenttype
Column removed|yes|Letter (Letter)|gcct_statusname
Column removed|yes|Letter (Letter)|gcct_parentactivity
Column removed|yes|Letter (Letter)|gcct_mailreachedstatus
Column removed|yes|Letter (Lett

Column removed|yes|Contact (Contact)|gcct_sourcemodifiedbyyominame
Column removed|yes|Contact (Contact)|gcct_timezonename
Column removed|yes|Contact (Contact)|gcct_imsystemname
New column added|yes|Contact (Contact)|address3_city
New column added|yes|Contact (Contact)|msdyn_disablewebtracking
New column added|yes|Contact (Contact)|address3_fax
Column removed|yes|Contact (Contact)|gcct_unformattedmobilephone
Column removed|yes|Contact (Contact)|gcct_donotsendmarketingmaterialsname
New column added|yes|Contact (Contact)|processid
New column added|yes|Contact (Contact)|address2_composite
New column added|yes|Contact (Contact)|slainvokedid
Column removed|yes|Contact (Contact)|gcct_facebookpage
Column removed|yes|Contact (Contact)|gcct_usedmercuryassigneddealer
Column removed|yes|Contact (Contact)|gcct_sourcemodifiedbyname
New column added|yes|Contact (Contact)|gcct_countryid
Column removed|yes|Contact (Contact)|gcct_last_upd_by
Column removed|yes|Contact (Contact)|gcct_sourcemodifiedby
New

New column added|yes|Contact (Contact)|gcct_customerdelegatephone
Column removed|yes|Contact (Contact)|gcct_contactidid
New column added|yes|Contact (Contact)|gcct_surveysuppressionsetbyuserid
New column added|yes|Contact (Contact)|address3_primarycontactname
Column removed|yes|Contact (Contact)|gcct_positionname
New column added|yes|Contact (Contact)|gcct_dealerportalrole
New column added|yes|Contact (Contact)|adx_profilealertinstructions
New column added|yes|Contact (Contact)|address3_addresstypecodename
Column removed|yes|Contact (Contact)|gcct_newlincolnassigneddealereffectivedate
Column removed|yes|Contact (Contact)|gcct_statusname
Column removed|yes|Contact (Contact)|gcct_suppressionend
New column added|yes|Contact (Contact)|gcct_espscore
dtype changed|yes|Contact (Contact)|gcct_inmarket|string|boolean
New column added|yes|Contact (Contact)|adx_identity_securitystamp
New column added|yes|Contact (Contact)|gcct_marketmailpreferenceupdatedbyidname
New column added|yes|Contact (Cont

New column added|yes|Appointment (Appointment)|gcct_dealerportal
New column added|yes|Appointment (Appointment)|isunsafe
New column added|yes|Appointment (Appointment)|gcct_phonefollowup
New column added|yes|Appointment (Appointment)|gcct_completewithoutactivityname
Column removed|yes|Appointment (Appointment)|gcct_creator_login
New column added|yes|Appointment (Appointment)|sortdate
New column added|yes|Appointment (Appointment)|gcct_completewithoutactivity
New column added|yes|Appointment (Appointment)|ismapiprivate
New column added|yes|Appointment (Appointment)|gcct_dm_source_record_id
New column added|yes|Appointment (Appointment)|activityadditionalparams
Column removed|yes|Goal (Goal)|gcct_accountgoalsidyominame
Column removed|yes|Goal (Goal)|gcct_accountgoalsid
New column added|yes|Goal (Goal)|entityimage_url
New column added|yes|Goal (Goal)|entityimage
Column removed|yes|Goal (Goal)|new_accountgoalsidname
Column removed|yes|Goal (Goal)|new_accountgoalsidyominame
Column removed|y

tables removed|may be|set([u'e-mail (email)', u'field involvement (fmc_fieldinv', u'document missed follow-up (fmc_', u'vehicle concern eci (fmc_vehicl', u'transfer escalation (fmc_transf', u'case resolution (incidentresolu', u'clp claim (fmc_clpclaim)', u'historical action origin (fmc_h', u'generic information (fmc_generi', u'certification catalog (fmc_cert', u'additional symptom (fmc_additio', u'resolution type (fmc_resolution', u'transmission (fmc_transmission)', u'customer market information (fm', u'marketing program (fmc_marketin', u'local issue (fmc_localissue)', u'roadside assistance (fmc_roadsi', u'master case (fmc_mastercase)', u'sme tar (fmc_smetechnicalassist', u'recall reimbursement plan (fmc_', u'sync ivt detail (fmc_syncivtdet', u'schedule follow-up (fmc_schedul', u'foc transportation assistance r', u'request vehicle (fmc_requestveh', u'activity party (activityparty)', u'dealer code (fmc_dealercode)', u'esp issue (fmc_espissues)', u'vehicle classification full pat', u'tes

In [12]:
columns_dtype_changed

[u'entitypicklist',
 u'gcct_customerinrental',
 u'gcct_vehicleoffroad',
 u'filterresultbystatus']

In [13]:
columns_added

[u'gcct_reassignifcaseownedbyidname',
 u'gcct_reassignifcaseownedbyid',
 u'gcct_assignmentfetchxml',
 u'gcct_activitytypename',
 u'gcct_activitytype',
 u'gcct_reassignifcaseownedbyidyominame',
 u'gcct_vhrreadydate',
 u'entitytypename',
 u'entitytype',
 u'entitytypename',
 u'entitytype',
 u'gcct_regardingid',
 u'gcct_regardingidname',
 u'navigationtype',
 u'prefix',
 u'filepointer',
 u'storagepointer',
 u'ismsteamssettingchangedbyuser',
 u'msdyn_orgchangestatus',
 u'msdyn_orgchangestatusname',
 u'gcct_dealerportalrole',
 u'azureactivedirectoryobjectid',
 u'ccof_dealeremail',
 u'ccof_dealername',
 u'ccof_dealeraddresscity',
 u'ccof_buyer',
 u'ccof_buyername',
 u'ccof_iscommercialaccount',
 u'ccof_dealeraddressstate',
 u'ccof_buyeridtype',
 u'ccof_buyeryominame',
 u'ccof_dealeraddresspostalcode',
 u'ccof_dealeraddressstreet',
 u'ccof_iscommercialaccountname',
 u'ccof_jitexpirationdatetime',
 u'ccof_dealertelephone',
 u'ccof_dealerfax',
 u'msdyn_logicalcollectionname',
 u'prefix',
 u'filep

In [14]:
columns_removed

[]

In [15]:
tables_added

[['s',
  'e',
  't',
  '(',
  '[',
  'u',
  "'",
  'm',
  'a',
  'r',
  'k',
  ' ',
  '(',
  'g',
  'c',
  'c',
  't',
  '_',
  'm',
  'a',
  'r',
  'k',
  'e',
  't',
  'i',
  'n',
  'g',
  'p',
  'r',
  'o',
  'g',
  'r',
  'a',
  'm',
  ')',
  "'",
  ',',
  ' ',
  'u',
  "'",
  'u',
  'p',
  'g',
  'r',
  'a',
  ' ',
  '(',
  'm',
  's',
  'd',
  'y',
  'n',
  '_',
  'u',
  'p',
  'g',
  'r',
  'a',
  'd',
  'e',
  'v',
  'e',
  'r',
  's',
  'i',
  'o',
  'n',
  ')',
  "'",
  ',',
  ' ',
  'u',
  "'",
  'b',
  'a',
  'c',
  ' ',
  't',
  'a',
  's',
  'k',
  ' ',
  '(',
  'g',
  'c',
  'c',
  't',
  '_',
  'b',
  'a',
  'c',
  't',
  'a',
  's',
  'k',
  ')',
  "'",
  ',',
  ' ',
  'u',
  "'",
  'e',
  'n',
  't',
  'i',
  't',
  'l',
  'e',
  'm',
  'e',
  'n',
  't',
  'e',
  'n',
  't',
  'i',
  't',
  'y',
  'a',
  'l',
  'l',
  'o',
  'c',
  'a',
  't',
  'i',
  'o',
  'n',
  't',
  'y',
  'p',
  'e',
  "'",
  ',',
  ' ',
  'u',
  "'",
  'u',
  'p',
  'g',
  'r',
  'a',
  'd',

In [16]:
tables_removed

[]